In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df.reset_index(inplace=True)

In [7]:
df.index

RangeIndex(start=0, stop=18285, step=1)

In [8]:
import tensorflow as tf

In [9]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/dhakad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [11]:
## Get the Dependent features
y=df['label']

In [12]:
X.shape, y.shape

((18285, 5), (18285,))

In [13]:
messages=X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
import re
from nltk.corpus import stopwords

In [16]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

### one_hot representation

In [19]:
voc_size = 5000

onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1545, 3681, 1575, 4337, 2972, 4326, 1223, 90, 2119, 2765],
 [3804, 3417, 3342, 4434, 1852, 1043, 4244],
 [1936, 67, 1137, 1155],
 [1844, 2154, 4284, 681, 4712, 927],
 [2927, 1852, 870, 3709, 3111, 3407, 1852, 3802, 4333, 2319],
 [3743,
  4126,
  3590,
  3444,
  3677,
  3789,
  3927,
  3769,
  2445,
  3259,
  2297,
  2924,
  664,
  999,
  4244],
 [2713, 2386, 4039, 1550, 4615, 2957, 2637, 3101, 3258, 2604, 889],
 [507, 1334, 3723, 4193, 1862, 1721, 3789, 503, 3258, 2604, 889],
 [2767, 4667, 3600, 3841, 1582, 1025, 1667, 3285, 3789, 4084],
 [1853, 2146, 4052, 4019, 428, 2887, 522, 4363],
 [2429, 1394, 1978, 1541, 4741, 4491, 2804, 4842, 4287, 3669, 4535],
 [681, 4276, 2972, 1025, 3789, 1862],
 [1491, 3442, 174, 3286, 4893, 1568, 4392, 1992, 2661],
 [1816, 1827, 1344, 3666, 1259, 2409, 3434, 3258, 2604, 889],
 [684, 4843, 996, 4808, 3326, 3258, 2604, 889],
 [1797, 3660, 3082, 448, 2253, 4708, 4113, 799, 2812, 4129],
 [657, 211, 3417],
 [785, 1395, 1736, 1797, 3789, 3378, 4693, 4244],
 [

### Embedding representation

In [20]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...   90 2119 2765]
 [   0    0    0 ... 1852 1043 4244]
 [   0    0    0 ...   67 1137 1155]
 ...
 [   0    0    0 ... 3258 2604  889]
 [   0    0    0 ...  948 3714 3747]
 [   0    0    0 ... 4308 1756 2432]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1545,
       3681, 1575, 4337, 2972, 4326, 1223,   90, 2119, 2765], dtype=int32)

In [25]:
## Creating model

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
len(embedded_docs),y.shape

(18285, (18285,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [30]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 16s 61ms/step - loss: 0.4410 - accuracy: 0.7504 - val_loss: 0.2007 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1237 - accuracy: 0.9536 - val_loss: 0.2063 - val_accuracy: 0.9150
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0774 - accuracy: 0.9729 - val_loss: 0.2572 - val_accuracy: 0.9135
Epoch 4/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0493 - accuracy: 0.9848 - val_loss: 0.2757 - val_accuracy: 0.9125
Epoch 5/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0324 - accuracy: 0.9895 - val_loss: 0.3246 - val_accuracy: 0.9127
Epoch 6/10
192/192 [==============================] - 5s 29ms/step - loss: 0.0206 - accuracy: 0.9930 - val_loss: 0.4547 - val_accuracy: 0.9153
Epoch 7/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0130 - accuracy: 0.9959 - val_loss: 0.4509 - val_accuracy: 0.916

In [31]:
# performance metrics

y_pred=model.predict_classes(X_test)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

/home/dhakad/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[3098,  321],
       [ 201, 2415]])

In [32]:
# accuracy

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9135045567522784

### To Remove Overfitting, Add Dropout

In [34]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [35]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 35ms/step - loss: 0.4812 - accuracy: 0.7477 - val_loss: 0.2005 - val_accuracy: 0.9178
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1482 - accuracy: 0.9436 - val_loss: 0.1943 - val_accuracy: 0.9185
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0946 - accuracy: 0.9664 - val_loss: 0.2151 - val_accuracy: 0.9203
Epoch 4/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0720 - accuracy: 0.9760 - val_loss: 0.2663 - val_accuracy: 0.9206
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0489 - accuracy: 0.9841 - val_loss: 0.2894 - val_accuracy: 0.9191
Epoch 6/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0350 - accuracy: 0.9880 - val_loss: 0.3497 - val_accuracy: 0.9152
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0309 - accuracy: 0.9896 - val_loss: 0.4059 - val_accuracy: 0.9158

In [36]:
# performance metrics

y_pred=model.predict_classes(X_test)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

/home/dhakad/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[3064,  355],
       [ 156, 2460]])

In [37]:
# accuracy

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9153272576636289